**Dependencies**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/curvature'

/content/drive/MyDrive/curvature


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.auto import tqdm, trange
import os
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
%matplotlib inline
import matplotlib.pyplot as plt

from Pointnet_model import Pointnet, loss_function
from modelnet40_data import PointnetDataset, load_data, PointSampler, Normalize, RandomNoise, RandomRotate, toTensor
from Pointnet_train_test import test

from visualization import pcshow

In [5]:
if torch.cuda.is_available():
      device = torch.device('cuda:0')
      print('running on GPU')
else:
      device = torch.device('cpu')
      print('running on CPU')

running on GPU


classes = {'airplane' : 0, 'bathtub' : 1, 'bed': 2, 'bench': 3, 'bookshelf':4, 'bottle':5, 'bowl' : 6, 'car':7, 'chair' : 8, 'cone' : 9, 'cup':10, 'curtain':11
           , 'desk':12, 'door':13, 'dresser':14, 'flower_pot':15, 'glass_box':16, 'guitar':17, 'keyboard':18, 'lamp':19, 'laptop':20, 'mantel':21, 'monitor':22,
           'nightstand':23, 'person':24, 'piano':25, 'plant':26, 'radio':27, 'range_hood':28, 'sink':29, 'sofa':30, 'stairs':31, 'stool':32, 'table':33, 'tent':34,
           'toilet':35, 'tv_stand':36, 'vase':37, 'wardrobe':38, 'xbox' : 39}

In [6]:
classes = {'airplane' : 0, 'bathtub' : 1, 'bed': 2, 'bench': 3, 'bookshelf':4, 'bottle':5, 'bowl' : 6, 'car':7, 'chair' : 8, 'cone' : 9, 'cup':10, 'curtain':11 , 'desk':12, 'door':13, 'dresser':14, 'flower_pot':15, 'glass_box':16, 'guitar':17, 'keyboard':18, 'lamp':19, 'laptop':20, 'mantel':21, 'monitor':22, 'nightstand':23, 'person':24, 'piano':25, 'plant':26, 'radio':27, 'range_hood':28, 'sink':29, 'sofa':30, 'stairs':31, 'stool':32, 'table':33, 'tent':34, 'toilet':35, 'tv_stand':36, 'vase':37, 'wardrobe':38, 'xbox' : 39}
res_classes = dict((v,k) for k,v in classes.items())

In [7]:
default_transform = transforms.Compose(
        [
            PointSampler(1024),
            Normalize(),
            toTensor()]
    )

# Create Dataset and Test

In [8]:
test_X, test_y = load_data(mode='test')

testset = PointnetDataset(test_X, test_y, transform=default_transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [9]:
model = Pointnet(device=device).to(device)
checkpoint = torch.load('checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])
test_loss, acc = test(model, testloader, device, verbose=True)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning:

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)



==> Test  | Average loss: 0.5079
==> Test  | Accuracy: 87.5608


In [10]:
for data in testloader:
  break

In [11]:
points, labels = data

In [12]:
def dynamic_saliency_map(model, points, labels, device, n= 100, T= 20, alpha= 1):
    model.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    points = points.transpose(1, 2).float() # B * 3 * 1024
    a = int(n/T)
    for i in range(T):
      points = points.to(device)
      labels = labels.to(device)
      # computing the gradient of the loss with respect to the points
      points.requires_grad = True
      o, rot3, rot64 = model(points)
      loss, _, _ = loss_function(o, labels, rot64, device)
      loss.backward()
      grad = points.grad
      # saliency map calculations
      sphere_core = torch.median(points, axis=2, keepdims=True) # B * 3 * 1

      sphere_r = torch.sqrt(torch.sum(torch.square(points - sphere_core.values), axis = 1)) # B * 1024

      sphere_axis = points - sphere_core.values # B * 3 * 1024

      before_map = torch.sum(torch.multiply(grad, sphere_axis), axis = 1) # B * 1024

      map =  -torch.multiply(before_map, torch.pow(sphere_r, alpha)) # B * 1024
      
      drop_indices = torch.topk(map, a).indices # B * n/T

      tmp = torch.zeros((points.shape[0], 3, points.shape[2] - a))
      numpy_points = points.cpu().detach().numpy()
      numpy_indices = drop_indices.cpu().detach().numpy()
      for j in range(points.shape[0]):
        tmp[j] = torch.from_numpy(np.delete(numpy_points[j], numpy_indices[j], axis  = 1))
      
      points = tmp.detach().clone() 
    return points

In [13]:
def correct_reshape(point):
  p = torch.zeros((point.shape[1], point.shape[0]))
  for i in range(point.shape[1]):
    p[i] = torch.from_numpy(np.array([point[0][i], point[1][i], point[2][i]]))
  return p

In [14]:
new_points = dynamic_saliency_map(model=model, points=points, labels=labels, device=device)

In [18]:
point_numbers = [1, 3, 6]
for i in range(len(point_numbers)):
  new_point = new_points[point_numbers[i]]
  new_point = correct_reshape(new_point)
  point = points[point_numbers[i]]
  print(res_classes[labels[point_numbers[i]].item()])
  pcshow(point[:,0], point[:,1], point[:,2], new_point[:, 0], new_point[:,1], new_point[:,2])

airplane


chair


toilet


In [ ]:
model.eval()
points = points.to(device)
labels = labels.to(device)
new_points = new_points.to(device)
o1, rot3, rot64 = model(points.reshape((32, 3, -1)).float())
_, predicted_1 = torch.max(o1.data, 1)
o2, rot3, rot64 = model(new_points.reshape((32, 3, -1)).float())
_, predicted_2 = torch.max(o2.data, 1)
correct1 = 0
correct1 += (labels == predicted_1).sum().item()
correct2 = 0
correct2 += (labels == predicted_2).sum().item()
print(correct1)
print(correct2)

31
21


In [ ]:
def setdiff2(p1, p2):
  new_len = len(p1) - len(p2)
  diff_arr = []
  diff = np.zeros((new_len, 3))
  for i in range(len(p1)):
    if p1[i] not in p2:
      diff_arr.append(p1[i])
  for i in range(len(diff_arr)):
    diff[i] = diff_arr[i]
  return diff